In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import random
from joblib import dump, load

In [2]:
# Fix random seed
seed = 42
np.random.seed(seed)
random.seed(seed)

# Pre-processing input data

In [3]:
def bits_to_MiB(row):
	# verify if has string ' MiB'
	if 'MiB' in str(row):
		row = row.replace(' MiB', '')
		row = float(row)
	else:
		row = float(row) / np.power(2, 20)
	return row


def MHz_to_GHz(row):
	# verify if has string ' GHz'
	if 'GHz' in str(row):
		row = row.replace(' GHz', '')
		# convert to float
		row = float(row)
	else:
		row = row.replace(' MHz', '')
		row = float(row) / 1000
	return row

In [4]:
results_df = pd.read_csv('../results_new/execution_time.csv')
results_savio_df = pd.read_csv('../results_savio_new/execution_time.csv')
results_df = pd.concat([results_df, results_savio_df], ignore_index=True)
# preprocessing
results_df['total_cpu_usage'] = results_df['total_cpu_usage'].str.replace('%', '').astype(float) / 100
results_df['max_ram_usage'] = results_df['max_ram_usage'] / 1024
results_df['l2_cache_size'] = results_df['l2_cache_size'].apply(bits_to_MiB)
results_df['l3_cache_size'] = results_df['l3_cache_size'].apply(bits_to_MiB)
results_df['ghz_actual_friendly'] = results_df['hz_actual_friendly'].apply(MHz_to_GHz)
results_df['ghz_advertised_friendly'] = results_df['hz_advertised_friendly'].str.replace('GHz', '').astype(float)
results_df = results_df.drop(columns=['hz_actual_friendly', 'hz_advertised_friendly', 'arch', 'vendor_id_raw'])

In [5]:
# Make the target dataset
target_df = results_df[['total_time', 'brand_raw', 'count', 'l2_cache_size', 'l3_cache_size', 'l2_cache_line_size', 'l2_cache_associativity', 'ghz_advertised_friendly', 'benchmark']].copy()
# Rename columns to *_target
target_df = target_df.rename(columns={
    'total_time': 'total_time_target',
    'brand_raw': 'brand_raw_target',
    'count': 'count_target',
    'l2_cache_size': 'l2_cache_size_target',
    'l3_cache_size': 'l3_cache_size_target',
    'l2_cache_line_size': 'l2_cache_line_size_target',
    'l2_cache_associativity': 'l2_cache_associativity_target',
    'ghz_advertised_friendly': 'ghz_advertised_friendly_target',
})

dataset_df = pd.merge(results_df, target_df, how='inner', on='benchmark')
dataset_df = dataset_df[dataset_df['brand_raw'] != dataset_df['brand_raw_target']]
dataset_df.head(2)

,total_time,total_cpu_usage,max_ram_usage,brand_raw,count,l2_cache_size,l3_cache_size,l2_cache_line_size,l2_cache_associativity,benchmark,ghz_actual_friendly,ghz_advertised_friendly,total_time_target,brand_raw_target,count_target,l2_cache_size_target,l3_cache_size_target,l2_cache_line_size_target,l2_cache_associativity_target,ghz_advertised_friendly_target
5,13.47,0.99,1436.714844,Intel(R) Core(TM) i5-10400 CPU @ 2.90GHz,12,1.5,12.0,256,6,KNP,4.1729,2.9,45.91,13th Gen Intel(R) Core(TM) i5-1335U,12,7.5,12.0,1280,7,2.496
6,13.47,0.99,1436.714844,Intel(R) Core(TM) i5-10400 CPU @ 2.90GHz,12,1.5,12.0,256,6,KNP,4.1729,2.9,25.77,13th Gen Intel(R) Core(TM) i5-1335U,12,7.5,12.0,1280,7,2.496


In [6]:
# remove one computer for testing
g_train = dataset_df[(dataset_df['brand_raw'] != '13th Gen Intel(R) Core(TM) i5-1335U') & (dataset_df['brand_raw_target'] != '13th Gen Intel(R) Core(TM) i5-1335U')]
g_test = dataset_df[dataset_df['brand_raw_target'] == '13th Gen Intel(R) Core(TM) i5-1335U']

In [7]:
mm_df = dataset_df[dataset_df['benchmark'].isin(['MATRIX_MULT', 'MATRIX_MULT2', 'MATRIX_MULT3'])]
# remove one computer for testing
mm_train = mm_df[(mm_df['brand_raw'] != '13th Gen Intel(R) Core(TM) i5-1335U') & (mm_df['brand_raw_target'] != '13th Gen Intel(R) Core(TM) i5-1335U')]
mm_test = mm_df[mm_df['brand_raw_target'] == '13th Gen Intel(R) Core(TM) i5-1335U']

In [8]:
st_df = dataset_df[~dataset_df['benchmark'].isin(['MATRIX_MULT', 'MATRIX_MULT2', 'MATRIX_MULT3'])]
# remove one computer for testing
st_train = st_df[(st_df['brand_raw'] != '13th Gen Intel(R) Core(TM) i5-1335U') & (st_df['brand_raw_target'] != '13th Gen Intel(R) Core(TM) i5-1335U')]
st_test = st_df[st_df['brand_raw_target'] == '13th Gen Intel(R) Core(TM) i5-1335U']

In [9]:
# load test dataset
# g_test = pd.read_csv('csv/g_test.csv')
# st_test = pd.read_csv('csv/st_test.csv')
# mm_test = pd.read_csv('csv/mm_test.csv')

In [10]:
target = 'total_time_target'
features = mm_test.columns.copy().drop(target).drop(['benchmark','brand_raw', 'brand_raw_target'])
features_st = features.copy().drop(['count', 'count_target'])

In [11]:
# general data
## split data
X_g_train = g_train[features]
y_g_train = g_train[target]

X_g_test = g_test[features]
y_g_test = g_test[target]

## normalize data
x_g_scaler = MinMaxScaler(feature_range=(0, 1))
X_g_train = x_g_scaler.fit_transform(X_g_train)
X_g_test = x_g_scaler.transform(X_g_test)
y_g_scaler = MinMaxScaler(feature_range=(0, 1))
y_g_train = y_g_scaler.fit_transform(y_g_train.values.reshape(-1, 1)).flatten()
y_g_test = y_g_scaler.transform(y_g_test.values.reshape(-1, 1)).flatten()

In [12]:
# single thread data
## split data
X_st_train = st_train[features_st]
y_st_train = st_train[target]

X_st_test = st_test[features_st]
y_st_test = st_test[target]

## normalize data
x_st_scaler = MinMaxScaler(feature_range=(0, 1))
X_st_train = x_st_scaler.fit_transform(X_st_train)
X_st_test = x_st_scaler.transform(X_st_test)
y_st_scaler = MinMaxScaler(feature_range=(0, 1))
y_st_train = y_st_scaler.fit_transform(y_st_train.values.reshape(-1, 1)).flatten()
y_st_test = y_st_scaler.transform(y_st_test.values.reshape(-1, 1)).flatten()

In [13]:
# multi thread data
## split data
X_mm_train = mm_train[features]
y_mm_train = mm_train[target]

X_mm_test = mm_test[features]
y_mm_test = mm_test[target]

## normalize data
x_mm_scaler = MinMaxScaler(feature_range=(0, 1))
X_mm_train = x_mm_scaler.fit_transform(X_mm_train)
X_mm_test = x_mm_scaler.transform(X_mm_test)
y_mm_scaler = MinMaxScaler(feature_range=(0, 1))
y_mm_train = y_mm_scaler.fit_transform(y_mm_train.values.reshape(-1, 1)).flatten()
y_mm_test = y_mm_scaler.transform(y_mm_test.values.reshape(-1, 1)).flatten()

# Training

In [14]:
models_folder = '../models/linear'
output_dim = 1

In [15]:
def inv_scaling(y, y_scaler):
    return y_scaler.inverse_transform(y.reshape(-1, 1))

In [16]:
#dump(scaler_g, f'{models_folder}/scaler_g.joblib')
#dump(scaler_st, f'{models_folder}/scaler_st.joblib')
#dump(scaler_mm, f'{models_folder}/scaler_mm.joblib')

## General

In [17]:
# general model initialization
model_g = LinearRegression()
model_g.fit(X_g_train, y_g_train)
preds = model_g.predict(X_g_test)
preds = inv_scaling(preds, y_g_scaler)
y_scaled = inv_scaling(y_g_test, y_g_scaler)
mse = mean_squared_error(y_scaled, preds)
mae = mean_absolute_error(y_scaled, preds)
print(f"MSE: {mse} - RMSE: {np.sqrt(mse)} - MAE: {mae}")

MSE: 407.7499505822544 - RMSE: 20.192819282662203 - MAE: 16.232445732029802


In [18]:
# save model
dump(model_g, f'{models_folder}/general.pt')

['../models/linear/general.pt']

## Single Thread

In [19]:
# single thread model initialization
model_st = LinearRegression()
model_st.fit(X_st_train, y_st_train)
preds = model_st.predict(X_st_test)
preds = inv_scaling(preds, y_st_scaler)
y_scaled = inv_scaling(y_st_test, y_st_scaler)
mse = mean_squared_error(y_scaled, preds)
mae = mean_absolute_error(y_scaled, preds)
print(f"MSE: {mse} - RMSE: {np.sqrt(mse)} - MAE: {mae}")

MSE: 260.1057267268679 - RMSE: 16.127793610003444 - MAE: 12.346330989856801


In [20]:
# save model
dump(model_st, f'{models_folder}/single_thread.pt')

['../models/linear/single_thread.pt']

## Multi Thread

In [21]:
# multi thread model initialization
model_mm = LinearRegression()
model_mm.fit(X_mm_train, y_mm_train)
preds = model_mm.predict(X_mm_test)
preds = inv_scaling(preds, y_mm_scaler)
y_scaled = inv_scaling(y_mm_test, y_mm_scaler)
mse = mean_squared_error(y_scaled, preds)
mae = mean_absolute_error(y_scaled, preds)
print(f"MSE: {mse} - RMSE: {np.sqrt(mse)} - MAE: {mae}")

MSE: 819.8171913750385 - RMSE: 28.632449971580122 - MAE: 28.614704128580044


In [22]:
# save model
dump(model_mm, f'{models_folder}/multi_thread.pt')

['../models/linear/multi_thread.pt']

# Load models

In [23]:
model_g = load(f'{models_folder}/general.pt')
model_st = load(f'{models_folder}/single_thread.pt')
model_mm = load(f'{models_folder}/multi_thread.pt')

In [24]:
def describe_val(model, X, y, y_scaler):
	min_instance = {"prediction": float('inf'), "actual": 0, "index": 0}
	max_instance = {"prediction": 0, "actual": 0, "index": 0}

	predictions = model.predict(X)
	predictions = inv_scaling(predictions, y_scaler)
	y_scaled = inv_scaling(y, y_scaler)
	index_min = np.argmin(np.abs(predictions - y_scaled))
	min_instance["prediction"] = predictions[index_min].item()
	min_instance["actual"] = y_scaled[index_min].item()
	min_instance["index"] = index_min
	index_max = np.argmax(np.abs(predictions - y_scaled))
	max_instance["prediction"] = predictions[index_max].item()
	max_instance["actual"] = y_scaled[index_max].item()
	max_instance["index"] = index_max

	return min_instance, max_instance, predictions

In [25]:
# general model
print("Validation set general model")
min_instance, max_instance, predictions = describe_val(model_g, X_g_test, y_g_test, y_g_scaler)
y_scaled = inv_scaling(y_g_test, y_g_scaler)
errors = np.abs(predictions - y_scaled)
mean_error = np.mean(errors)
std_error = np.std(errors)

print(f"Mean prediction: {np.mean(predictions)} | Std actual: {np.std(predictions)}")
print(f"Mean actual: {np.mean(y_scaled)} | Std actual: {np.std(y_scaled)}")
print(f"Mean Error: {mean_error} | Std Error: {std_error}")
print("---")
print("Min instance")
print(g_test.iloc[min_instance["index"]])
print(f"Min Prediction: {min_instance['prediction']} | Actual: {min_instance['actual']} | Error: {abs(min_instance['prediction'] - min_instance['actual'])}")
print("---")
print("Max instance")
print(g_test.iloc[max_instance["index"]])
print(f"Max Prediction: {max_instance['prediction']} | Actual: {max_instance['actual']} | Error: {abs(max_instance['prediction'] - max_instance['actual'])}")

Validation set general model
Mean prediction: 18.82346850249339 | Std actual: 22.19378145202373
Mean actual: 35.041486486486484 | Std actual: 28.83706408578472
Mean Error: 16.232445732029802 | Std Error: 12.010730874470624
---
Min instance
total_time                                                           33.51
total_cpu_usage                                                       0.99
max_ram_usage                                                  1435.382812
brand_raw                         Intel(R) Core(TM) i5-8300H CPU @ 2.30GHz
count                                                                    8
l2_cache_size                                                          1.0
l3_cache_size                                                          8.0
l2_cache_line_size                                                     256
l2_cache_associativity                                                   6
benchmark                                                              KNP
ghz_actual

In [26]:
# single thread model
print("Validation set single thread model")
min_instance, max_instance, predictions = describe_val(model_st, X_st_test, y_st_test, y_st_scaler)
y_scaled = inv_scaling(y_st_test, y_st_scaler)
errors = np.abs(predictions - y_scaled)
mean_error = np.mean(errors)
std_error = np.std(errors)

print(f"Mean prediction: {np.mean(predictions)} | Std actual: {np.std(predictions)}")
print(f"Mean actual: {np.mean(y_scaled)} | Std actual: {np.std(y_scaled)}")
print(f"Mean Error: {mean_error} | Std Error: {std_error}")
print("Min instance")
print(st_test.iloc[min_instance["index"]])
print(f"Min Prediction: {min_instance['prediction']} | Actual: {min_instance['actual']} | Error: {abs(min_instance['prediction'] - min_instance['actual'])}")
print("---")
print("Max instance")
print(st_test.iloc[max_instance["index"]])
print(f"Max Prediction: {max_instance['prediction']} | Actual: {max_instance['actual']} | Error: {abs(max_instance['prediction'] - max_instance['actual'])}")

Validation set single thread model
Mean prediction: 22.476520931183096 | Std actual: 23.89007554618783
Mean actual: 34.75261290322581 | Std actual: 31.49396398818138
Mean Error: 12.346330989856801 | Std Error: 10.376600494177728
Min instance
total_time                                                            77.73
total_cpu_usage                                                        0.99
max_ram_usage                                                      78.28125
brand_raw                         Intel(R) Xeon(R) CPU E5-2650 v4 @ 2.20GHz
count                                                                    24
l2_cache_size                                                           6.0
l3_cache_size                                                          30.0
l2_cache_line_size                                                      256
l2_cache_associativity                                                    6
benchmark                                                              TSP

In [27]:
# multi thread model
print("Validation set multi thread model")
min_instance, max_instance, predictions = describe_val(model_mm, X_mm_test, y_mm_test, y_mm_scaler)
y_scaled = inv_scaling(y_mm_test, y_mm_scaler)
errors = np.abs(predictions - y_scaled)
mean_error = np.mean(errors)
std_error = np.std(errors)

print(f"Mean prediction: {np.mean(predictions)} | Std actual: {np.std(predictions)}")
print(f"Mean actual: {np.mean(y_scaled)} | Std actual: {np.std(y_scaled)}")
print(f"Mean Error: {mean_error} | Std Error: {std_error}")
print("Min instance")
print(mm_test.iloc[min_instance["index"]])
print(f"Min Prediction: {min_instance['prediction']} | Actual: {min_instance['actual']} | Error: {abs(min_instance['prediction'] - min_instance['actual'])}")
print("---")
print("Max instance")
print(mm_test.iloc[max_instance["index"]])
print(f"Max Prediction: {max_instance['prediction']} | Actual: {max_instance['actual']} | Error: {abs(max_instance['prediction'] - max_instance['actual'])}")

Validation set multi thread model
Mean prediction: 7.919295871419956 | Std actual: 0.5305421838672161
Mean actual: 36.534 | Std actual: 0.8569854141115829
Mean Error: 28.614704128580044 | Std Error: 1.0079181558353814
Min instance
total_time                                                            3.47
total_cpu_usage                                                      18.37
max_ram_usage                                                  2423.492188
brand_raw                         Intel(R) Xeon(R) Gold 6130 CPU @ 2.10GHz
count                                                                   32
l2_cache_size                                                         32.0
l3_cache_size                                                         22.0
l2_cache_line_size                                                     256
l2_cache_associativity                                                   6
benchmark                                                      MATRIX_MULT
ghz_actual_friendly